## Завдання: Task #2 Модель визначення вікової категорії

- Бізнес ціль: Визначення вікової категорії для доповнення портрету абонентів-користувачів послуг та для використання в рамках пропозицій цільового маркетингу
- Задача: Визначити вікову категорію абонентів

### Model Target

- 1 – Age group <20 years
- 2 – Age group 20-30 years
- 3 – Age group 30-40 years
- 4 – Age group 40-50 years
- 5 – Age group >50 years

### Sample size

- Train 77% (147К)
- Test 23% (44K)

### Дані

- TRAIN_AGE_MODEL_FE (Target - розмітка ) – абонентська вітрина
- TRAIN_AGE_MODEL_B_NUM – дані про короткі номери
- TRAIN_AGE_MODEL_DPI – трафік мобільних додатків

Опис даних в файлі «Описание данных.xlsx»

### Evaluation

Метриками якості моделі визначені показники AUC, Classification report, Confusion matrix на Test Sample.
За базову метрику для порівняння якості моделей студентів обрана Accuracy з базовим результатом
(Baseline model) 0,45 на Test Sample.

### Результат роботи

Результатом роботи є Jupyter Notebook з коментарями до коду, за допомогою якого
прогнозується результат на Test Sample та презентації (графіки, таблиці) результатів з
орієнтацією на бізнес (не технічних) користувачів, що має включати наступні пункти: feature
explorer, feature encoding, спосіб валідації, best model (best algorithm/best metrics), підхід до
підбору hyperparameters model, features importance, shap


In [17]:
import pickle

with open("./age_model_pd2.pcl", "rb") as f:
    data = pickle.load(f)

data_names = list(data[1][0].values()) + list(data[2][0].values())
data = data[1][1:] + data[2][1:]

for index, name in enumerate(data_names):
    print(f"{name}: {data[index].shape}")

TRAIN_MODEL_FE: (146953, 817)
TRAIN_MODEL_B_NUM: (803268, 8)
TRAIN_MODEL_DPI: (7058446, 6)
TEST_MODEL_FE: (44307, 817)
TEST_MODEL_B_NUM: (244897, 8)
TEST_MODEL_DPI: (2133277, 6)


`B_NUM` and `DPI` are many to one (one `abon_id` can have multiple num and dpi entries). Therefore we can not just merge this data in one df.


In [18]:
# unpack the data
from utils import compress_df
import dask.dataframe as dd

df_train_fe, df_train_num, df_train_dpi, df_test_fe, df_test_num, df_test_dpi = map(
    lambda df: dd.from_pandas(compress_df(df))
    .set_index("abon_id")
    .repartition(npartitions=2),
    data,
)
del data

TARGET_KEY = "target"
df_train_fe[TARGET_KEY] = df_train_fe[TARGET_KEY].astype("int8")
df_test_fe[TARGET_KEY] = df_test_fe[TARGET_KEY].astype("int8")

Mem. usage decreased to 238.53 Mb (73.9% reduction)
Mem. usage decreased to 18.39 Mb (60.0% reduction)
Mem. usage decreased to 94.24 Mb (65.0% reduction)
Mem. usage decreased to 71.92 Mb (73.9% reduction)
Mem. usage decreased to  5.61 Mb (60.0% reduction)
Mem. usage decreased to 28.48 Mb (65.0% reduction)


In [14]:
import shutil

shutil.rmtree('./data', ignore_errors=True)

df_dict = {
    "train_fe": df_train_fe,
    "train_num": df_train_num,
    "train_dpi": df_train_dpi,
    "test_fe": df_test_fe,
    "test_num": df_test_num,
    "test_dpi": df_test_dpi,
}

for key, df in df_dict.items():
    df.to_parquet(f"./data/{key}")